## Imports and initialization

In [1]:
import os
import json

from dataverk import Client
from dataverk_vault import api as vault_api
from google.cloud import storage
from google.oauth2 import service_account

### Dataverk

In [2]:
dv = Client()
vault_api.set_secrets_as_envs()

### Google cloud

In [3]:
creds = service_account.Credentials.from_service_account_info(
                json.loads(os.environ["GOOGLE_CREDS"]), scopes=("https://www.googleapis.com/auth/cloud-platform",)
            )

In [4]:
client = storage.Client(creds.project_id, credentials=creds)

In [5]:
bucket = client.get_bucket("styrk-bucket")

## Read from database

In [6]:
df = dv.read_sql("datalab", "SELECT * FROM dataverk.mytable")

INFO:jovyan:OracleConnector:2021-01-07 11:06:20: Reading from database: datalab
/opt/conda/lib/python3.7/site-packages/sqlalchemy/dialects/oracle/base.py:1380: SAWarning: Oracle version (12, 2, 0, 1, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))
INFO:jovyan:OracleConnector:2021-01-07 11:06:20: 6 records returned in 0.06449508666992188 seconds.


In [7]:
df.shape

(6, 3)

## Convert dataframe to pickle and upload to gcs

In [8]:
df.to_pickle("databasedump.pkl")

In [9]:
blob = bucket.blob("airflow/databasedump.pkl")

In [10]:
blob.upload_from_filename("databasedump.pkl")